In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

# ===============================
# 0. Configuración de rutas
# ===============================
# Carpeta donde se guardan los gráficos de INICIAL
RESULTS_DIR = r"C:\Users\Hp\Desktop\WebScraping\Personal\tailoy-ipc-webscraping\resultados\educacion_inicial"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Archivo consolidado de educación
INPUT_FILE = r"C:\Users\Hp\Desktop\WebScraping\Personal\tailoy-ipc-webscraping\normalized_consolidated\consolidado_educacion.xlsx"

# ===============================
# 1. Cargar datos
# ===============================
df = pd.read_excel(INPUT_FILE)

# Asegurar que el precio esté en formato numérico
df["precio_por_unidad_estandar"] = pd.to_numeric(
    df["precio_por_unidad_estandar"], errors="coerce"
)

# Eliminar filas sin precio válido
df = df.dropna(subset=["precio_por_unidad_estandar"]).copy()

# ===============================
# 2. Filtrar SOLO productos de INICIAL
# ===============================
productos_inicial = [
    "cartulinas",
    "colores",
    "crayones-y-oleos",
    "cuaderno triple reglon",
    "limpiatipos",
    # "loncheras",
    "plastilinas",
    "reglas",
]

df["producto_especifico"] = df["producto_especifico"].astype(str)
df = df[df["producto_especifico"].isin(productos_inicial)].copy()

# (opcional) marcar que todo es INICIAL
df["nivel_educativo"] = "INICIAL"

# ===============================
# 3. Preparar fecha
# ===============================
if "fecha_scraping" in df.columns:
    df["FECHA"] = pd.to_datetime(df["fecha_scraping"], errors="coerce")
else:
    df["FECHA"] = pd.NaT

# ===============================
# 4. Volatilidad por producto (solo INICIAL, precio directo)
# ===============================
vol_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .std()
      .reset_index(name="VOLATILIDAD")
)

print("Volatilidad por producto - INICIAL (precio directo):")
print(vol_prod)

# Heatmap: una fila con N productos
fig_vol = go.Figure(
    data=go.Heatmap(
        z=[vol_prod["VOLATILIDAD"].values],      # una fila con N productos
        x=vol_prod["producto_especifico"],       # eje X: productos
        y=["Volatilidad"],                       # eje Y: una sola fila
        colorscale="YlOrRd",
        colorbar=dict(title="Volatilidad"),
    )
)

# Anotar valores dentro de cada celda
for _, row in vol_prod.iterrows():
    fig_vol.add_trace(
        go.Scatter(
            x=[row["producto_especifico"]],
            y=["Volatilidad"],
            text=[f"{row['VOLATILIDAD']:.2f}"],
            mode="text",
            showlegend=False,
        )
    )

fig_vol.update_layout(
    title=None,
    xaxis_title="Producto (INICIAL)",   # sin 'específico'
    yaxis_title="",
    template="plotly_white",
    width=1200,
    height=400,
    margin=dict(l=40, r=40, t=40, b=80),
)
fig_vol.show()
fig_vol.write_image(
    os.path.join(RESULTS_DIR, "volatilidad_por_producto_inicial.pdf")
)

# ===============================
# 5. Boxplot de precios por producto (precio directo)
# ===============================
fig_box = px.box(
    df,
    x="producto_especifico",
    y="precio_por_unidad_estandar",
    color="producto_especifico",
    labels={
        "producto_especifico": "Producto (INICIAL)",
        "precio_por_unidad_estandar": "Precio por unidad estándar (S/)",
    },
    width=1400,
    height=650,
    template="plotly_white",
)

fig_box.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,  # la leyenda sería igual que el eje X
    title=None,
    margin=dict(l=40, r=40, t=40, b=120),
)
fig_box.show()
fig_box.write_image(
    os.path.join(RESULTS_DIR, "boxplot_precio_por_producto_inicial.pdf")
)

# ===============================
# 6. Evolución del precio promedio por producto (precio directo)
# ===============================
if df["FECHA"].notna().any():
    evol_prod = (
        df.groupby(["FECHA", "producto_especifico"])["precio_por_unidad_estandar"]
          .mean()
          .reset_index()
    )

    fig_line = px.line(
        evol_prod,
        x="FECHA",
        y="precio_por_unidad_estandar",
        color="producto_especifico",
        markers=True,
        labels={
            "FECHA": "Fecha",
            "precio_por_unidad_estandar": "Precio promedio (S/)",
            "producto_especifico": "Producto (INICIAL)",
        },
    )

    fig_line.update_layout(
        title=None,
        xaxis_title="Fecha",
        yaxis_title="Precio promedio (S/)",
        template="plotly_white",
        hovermode="x unified",
        width=900,
        height=500,
        legend=dict(
            orientation="h",       # horizontal
            yanchor="top",
            y=-0.30,               # leyenda debajo del gráfico
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
        ),
        margin=dict(l=40, r=40, t=40, b=120),
    )

    fig_line.show()
    fig_line.write_image(
        os.path.join(RESULTS_DIR, "evolucion_precio_por_producto_inicial.pdf")
    )
else:
    print("No hay 'fecha_scraping' válida para hacer la evolución temporal.")


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

# ===============================
# 0. Configuración de rutas
# ===============================
# Carpeta donde se guardan los gráficos de PRIMARIA
RESULTS_DIR = r"C:\Users\Hp\Desktop\WebScraping\Personal\tailoy-ipc-webscraping\resultados\educacion_primaria"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Archivo consolidado de educación
INPUT_FILE = r"C:\Users\Hp\Desktop\WebScraping\Personal\tailoy-ipc-webscraping\normalized_consolidated\consolidado_educacion.xlsx"

# ===============================
# 1. Cargar datos
# ===============================
df = pd.read_excel(INPUT_FILE)

# Asegurar que el precio esté en formato numérico
df["precio_por_unidad_estandar"] = pd.to_numeric(
    df["precio_por_unidad_estandar"], errors="coerce"
)

# Eliminar filas sin precio válido
df = df.dropna(subset=["precio_por_unidad_estandar"]).copy()

# ===============================
# 2. Filtrar SOLO productos de PRIMARIA
# ===============================
productos_primaria = [
    "borradores",
    "colas",
    "cuaderno doble reglon",
    "cuaderno rayado",
    "forros",
    "gomas",
    "lapices",
    "tajadores",
    "tijeras",
]

df["producto_especifico"] = df["producto_especifico"].astype(str)
df = df[df["producto_especifico"].isin(productos_primaria)].copy()

# (opcional) marcar que todo es PRIMARIA
df["nivel_educativo"] = "PRIMARIA"

# ===============================
# 3. Preparar fecha
# ===============================
if "fecha_scraping" in df.columns:
    df["FECHA"] = pd.to_datetime(df["fecha_scraping"], errors="coerce")
else:
    df["FECHA"] = pd.NaT

# ===============================
# 4. Volatilidad por producto (solo PRIMARIA)
# ===============================
vol_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .std()
      .reset_index(name="VOLATILIDAD")
)

print("Volatilidad por producto - PRIMARIA (precio directo):")
print(vol_prod)

# Heatmap: una fila con N productos
fig_vol = go.Figure(
    data=go.Heatmap(
        z=[vol_prod["VOLATILIDAD"].values],      # una fila con N productos
        x=vol_prod["producto_especifico"],       # eje X: productos
        y=["Volatilidad"],                       # eje Y: una sola fila
        colorscale="YlOrRd",
        colorbar=dict(title="Volatilidad"),
    )
)

# Anotar valores dentro de cada celda
for _, row in vol_prod.iterrows():
    fig_vol.add_trace(
        go.Scatter(
            x=[row["producto_especifico"]],
            y=["Volatilidad"],
            text=[f"{row['VOLATILIDAD']:.2f}"],
            mode="text",
            showlegend=False,
        )
    )

fig_vol.update_layout(
    title=None,
    xaxis_title="Producto (PRIMARIA)",   # sin 'específico'
    yaxis_title="",
    template="plotly_white",
    width=1200,
    height=400,
    margin=dict(l=40, r=40, t=40, b=80),
)
fig_vol.show()
fig_vol.write_image(
    os.path.join(RESULTS_DIR, "volatilidad_por_producto_primaria.pdf")
)

# ===============================
# 5. Boxplot de precios por producto (precio directo)
# ===============================
fig_box = px.box(
    df,
    x="producto_especifico",
    y="precio_por_unidad_estandar",
    color="producto_especifico",
    labels={
        "producto_especifico": "Producto (PRIMARIA)",         
        "precio_por_unidad_estandar": "Precio por unidad estándar (S/)",
    },
    width=1400,
    height=650,
    template="plotly_white",
)

fig_box.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,  # la leyenda sería igual que el eje X
    title=None,
    margin=dict(l=40, r=40, t=40, b=120),
)
fig_box.show()
fig_box.write_image(
    os.path.join(RESULTS_DIR, "boxplot_precio_por_producto_primaria.pdf")
)

# ===============================
# 6. Evolución del precio promedio por producto (precio directo)
# ===============================
if df["FECHA"].notna().any():
    evol_prod = (
        df.groupby(["FECHA", "producto_especifico"])["precio_por_unidad_estandar"]
          .mean()
          .reset_index()
    )

    fig_line = px.line(
        evol_prod,
        x="FECHA",
        y="precio_por_unidad_estandar",
        color="producto_especifico",
        markers=True,
        labels={
            "FECHA": "Fecha",
            "precio_por_unidad_estandar": "Precio promedio (S/)",
            "producto_especifico": "Producto (PRIMARIA)",
        },
    )

    fig_line.update_layout(
        title=None,
        xaxis_title="Fecha",
        yaxis_title="Precio promedio (S/)",
        template="plotly_white",
        hovermode="x unified",
        width=900,
        height=500,
        legend=dict(
            orientation="h",       # horizontal
            yanchor="top",
            y=-0.30,               # leyenda debajo del gráfico
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
        ),
        margin=dict(l=40, r=40, t=40, b=120),
    )

    fig_line.show()
    fig_line.write_image(
        os.path.join(RESULTS_DIR, "evolucion_precio_por_producto_primaria.pdf")
    )
else:
    print("No hay 'fecha_scraping' válida para hacer la evolución temporal.")


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os

# ===============================
# 0. Configuración de rutas
# ===============================
# Carpeta donde se guardan los gráficos de SECUNDARIA
RESULTS_DIR = r"C:\Users\Hp\Desktop\WebScraping\Personal\tailoy-ipc-webscraping\resultados\educacion_secundaria"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Archivo consolidado de educación
INPUT_FILE = r"C:\Users\Hp\Desktop\WebScraping\Personal\tailoy-ipc-webscraping\normalized_consolidated\consolidado_educacion.xlsx"

# ===============================
# 1. Cargar datos
# ===============================
df = pd.read_excel(INPUT_FILE)

# Asegurar que el precio esté en formato numérico
df["precio_por_unidad_estandar"] = pd.to_numeric(
    df["precio_por_unidad_estandar"], errors="coerce"
)

# Eliminar filas sin precio válido
df = df.dropna(subset=["precio_por_unidad_estandar"]).copy()

# ===============================
# 2. Filtrar SOLO productos de SECUNDARIA
# ===============================
productos_secundaria = [
    # "calculadoras",
    "compases",
    "correctores",
    "cuaderno cuadriculado",
    "escuadras",
    "lapiceros",
    # "mochila escolar",
    "papel-bond",
    "plumones-para-papel",
    "resaltadores",
]

df["producto_especifico"] = df["producto_especifico"].astype(str)
df = df[df["producto_especifico"].isin(productos_secundaria)].copy()

# (opcional) marcar que todo es SECUNDARIA
df["nivel_educativo"] = "SECUNDARIA"

# ===============================
# 3. Preparar fecha
# ===============================
if "fecha_scraping" in df.columns:
    df["FECHA"] = pd.to_datetime(df["fecha_scraping"], errors="coerce")
else:
    df["FECHA"] = pd.NaT

# ===============================
# 4. Volatilidad por producto (SECUNDARIA, precio directo)
# ===============================
vol_prod = (
    df.groupby("producto_especifico")["precio_por_unidad_estandar"]
      .std()
      .reset_index(name="VOLATILIDAD")
)

print("Volatilidad por producto - SECUNDARIA (precio directo):")
print(vol_prod)

# Heatmap: una fila con N productos
fig_vol = go.Figure(
    data=go.Heatmap(
        z=[vol_prod["VOLATILIDAD"].values],      # una fila con N productos
        x=vol_prod["producto_especifico"],       # eje X: productos
        y=["Volatilidad"],                       # eje Y: una sola fila
        colorscale="YlOrRd",
        colorbar=dict(title="Volatilidad"),
    )
)

# Anotar valores dentro de cada celda
for _, row in vol_prod.iterrows():
    fig_vol.add_trace(
        go.Scatter(
            x=[row["producto_especifico"]],
            y=["Volatilidad"],
            text=[f"{row['VOLATILIDAD']:.2f}"],
            mode="text",
            showlegend=False,
        )
    )

fig_vol.update_layout(
    title=None,
    xaxis_title="Producto (SECUNDARIA)",   # sin 'específico'
    yaxis_title="",
    template="plotly_white",
    width=1200,
    height=400,
    margin=dict(l=40, r=40, t=40, b=80),
)
fig_vol.show()
fig_vol.write_image(
    os.path.join(RESULTS_DIR, "volatilidad_por_producto_secundaria.pdf")
)

# ===============================
# 5. Boxplot de precios por producto (precio directo)
# ===============================
fig_box = px.box(
    df,
    x="producto_especifico",
    y="precio_por_unidad_estandar",
    color="producto_especifico",
    labels={
        "producto_especifico": "Producto (SECUNDARIA)",
        "precio_por_unidad_estandar": "Precio por unidad estándar (S/)",
    },
    width=1400,
    height=650,
    template="plotly_white",
)

fig_box.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,  # la leyenda sería igual que el eje X
    title=None,
    margin=dict(l=40, r=40, t=40, b=120),
)
fig_box.show()
fig_box.write_image(
    os.path.join(RESULTS_DIR, "boxplot_precio_por_producto_secundaria.pdf")
)

# ===============================
# 6. Evolución del precio promedio por producto (precio directo)
# ===============================
if df["FECHA"].notna().any():
    evol_prod = (
        df.groupby(["FECHA", "producto_especifico"])["precio_por_unidad_estandar"]
          .mean()
          .reset_index()
    )

    fig_line = px.line(
        evol_prod,
        x="FECHA",
        y="precio_por_unidad_estandar",
        color="producto_especifico",
        markers=True,
        labels={
            "FECHA": "Fecha",
            "precio_por_unidad_estandar": "Precio promedio (S/)",
            "producto_especifico": "Producto (SECUNDARIA)",
        },
    )

    fig_line.update_layout(
        title=None,
        xaxis_title="Fecha",
        yaxis_title="Precio promedio (S/)",
        template="plotly_white",
        hovermode="x unified",
        width=900,
        height=500,
        legend=dict(
            orientation="h",       # horizontal
            yanchor="top",
            y=-0.30,               # leyenda debajo del gráfico
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
        ),
        margin=dict(l=40, r=40, t=40, b=120),
    )

    fig_line.show()
    fig_line.write_image(
        os.path.join(RESULTS_DIR, "evolucion_precio_por_producto_secundaria.pdf")
    )
else:
    print("No hay 'fecha_scraping' válida para hacer la evolución temporal.")
